# Agents

Agent = large language model (LLM), configured with instructions and tools.

Basic configuration:

- **name**: A required string that identifies your agent.
- **instructions**: also known as a developer message or system prompt.
- **model**: which LLM to use, and optional model_settings to configure model tuning parameters like - temperature, top_p, etc.
- **tools**: Tools that the agent can use to achieve its tasks.

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from agents import Agent, ModelSettings, function_tool

@function_tool
def get_weather(city: str) -> str:
     """returns weather info for the specified city."""
     return f"The weather in {city} is sunny"


agent = Agent(
    name="Weather agent",
    instructions="You use the get_weather tool to get the weather in a city.",
    model="gpt-5-mini",
    tools=[get_weather],
)

In [7]:
from agents import Runner

result = Runner.run_sync(agent, 'What is the weather in Tokyo?')

In [8]:
print(result.final_output)

The weather in Tokyo is sunny.


# Live Example Structured Outputs

In [10]:
from pydantic import BaseModel, Field
from agents.tool import WebSearchTool

class ProductInfo(BaseModel):
    product_name: str = Field(description="The name of the product")
    product_price: float = Field(description="The price of the product")
    product_description: str = Field(description="A description of the product")

agent = Agent(
    name="Product Search Agent",
    instructions="You use the WebSearchTool to search the web for product information.",
    model="gpt-5-mini",
    tools=[WebSearchTool()],
    output_type=ProductInfo,
)

result = Runner.run_sync(agent, 'Research an USBC to ethernet adapter on amazon or others')

print(result.final_output)

product_name='Cable Matters USB C to 2.5 Gigabit Ethernet Adapter with Charging (100W)' product_price=35.0 product_description='Summary: 2.5 Gbps USB‑C to RJ45 adapter with pass‑through Power Delivery (up to 100W). Good balance of speed, build quality, and price for laptops that need faster-than‑Gigabit wired networking. \ue200cite\ue202turn0search0\ue201\n\nKey specs & notes:\n- Network speed: up to 2.5 Gbps (backward compatible with 10/100/1000 Mbps). \ue200cite\ue202turn0search0\ue201\n- Power delivery: up to 100W passthrough (can charge your laptop while using Ethernet). \ue200cite\ue202turn0search0\ue201\n- Driver/compatibility: plug‑and‑play on many systems, but the listing notes additional drivers may be required to reach full 2.5 Gbps on some OS/hardware — check your host OS and chipset. \ue200cite\ue202turn0search0\ue201\n- Price (Amazon listing): approximately $35 (check current listing for final price/availability). \ue200cite\ue202turn0search0\ue201\n\nBuying tips:\n- To ac

In [12]:
result.final_output.product_name

'Cable Matters USB C to 2.5 Gigabit Ethernet Adapter with Charging (100W)'

In [13]:
result.final_output.product_description

'Summary: 2.5 Gbps USB‑C to RJ45 adapter with pass‑through Power Delivery (up to 100W). Good balance of speed, build quality, and price for laptops that need faster-than‑Gigabit wired networking. \ue200cite\ue202turn0search0\ue201\n\nKey specs & notes:\n- Network speed: up to 2.5 Gbps (backward compatible with 10/100/1000 Mbps). \ue200cite\ue202turn0search0\ue201\n- Power delivery: up to 100W passthrough (can charge your laptop while using Ethernet). \ue200cite\ue202turn0search0\ue201\n- Driver/compatibility: plug‑and‑play on many systems, but the listing notes additional drivers may be required to reach full 2.5 Gbps on some OS/hardware — check your host OS and chipset. \ue200cite\ue202turn0search0\ue201\n- Price (Amazon listing): approximately $35 (check current listing for final price/availability). \ue200cite\ue202turn0search0\ue201\n\nBuying tips:\n- To actually get 2.5 Gbps you also need a 2.5G‑capable switch/router and appropriate Cat5e/Cat6 cable; otherwise the adapter will fal

In [14]:
result.final_output.product_price

35.0

In [15]:
import pandas as pd

# Create a DataFrame from the result
df = pd.DataFrame([{
    "Product Name": result.final_output.product_name,
    "Price (USD)": result.final_output.product_price,
    "Description": result.final_output.product_description
}])

# Display the DataFrame as a nice table
from IPython.display import display
display(df)


,Product Name,Price (USD),Description
0,Cable Matters USB C to 2.5 Gigabit Ethernet Ad...,35.0,Summary: 2.5 Gbps USB‑C to RJ45 adapter with p...


In [21]:
result.final_output.model_dump_json()

'{"product_name":"Cable Matters USB C to 2.5 Gigabit Ethernet Adapter with Charging (100W)","product_price":35.0,"product_description":"Summary: 2.5 Gbps USB‑C to RJ45 adapter with pass‑through Power Delivery (up to 100W). Good balance of speed, build quality, and price for laptops that need faster-than‑Gigabit wired networking. \ue200cite\ue202turn0search0\ue201\\n\\nKey specs & notes:\\n- Network speed: up to 2.5 Gbps (backward compatible with 10/100/1000 Mbps). \ue200cite\ue202turn0search0\ue201\\n- Power delivery: up to 100W passthrough (can charge your laptop while using Ethernet). \ue200cite\ue202turn0search0\ue201\\n- Driver/compatibility: plug‑and‑play on many systems, but the listing notes additional drivers may be required to reach full 2.5 Gbps on some OS/hardware — check your host OS and chipset. \ue200cite\ue202turn0search0\ue201\\n- Price (Amazon listing): approximately $35 (check current listing for final price/availability). \ue200cite\ue202turn0search0\ue201\\n\\nBuyin

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio

from agents import Agent, Runner, WebSearchTool, trace



agent = Agent(
    name="Web searcher",
    instructions="You are a helpful agent.",
    tools=[WebSearchTool(user_location={"type": "approximate", "city": "Lisbon"})],
)

with trace("Web search example"):
    result = await Runner.run(
        agent,
        "Search the best fridges to buy in Lisbon in 2025.",
    )
    print(result.final_output)

When selecting a refrigerator in Lisbon in 2025, it's essential to consider factors such as energy efficiency, capacity, design, and advanced features. Here are some top-rated models to consider:



**Haier Twin Inverter Refrigerator**  
Recognized as the most trusted refrigerator brand in Portugal in 2025, Haier offers models with Twin Inverter Technology, ensuring quick cooling and reduced noise. ([corporate.haier-europe.com](https://corporate.haier-europe.com/press-release/haier-elected-as-the-most-trusted-brand-in-refrigerators-in-portugal/?utm_source=openai))




**LG InstaView Door-in-Door Refrigerator**  
This model features a sleek design with a glass panel that illuminates with two knocks, allowing you to see inside without opening the door, thereby conserving energy. ([analyticsinsight.net](https://www.analyticsinsight.net/home-electronics/top-10-refrigerators-for-large-families-in-2025-with-smart-cooling?utm_source=openai))




**Samsung Family Hub Refrigerator**  
Equipped 

In [27]:
import os
from pathlib import Path
from typing import List, Dict
from agents import function_tool

@function_tool
def search_course_notes(query: str) -> Dict[str, List[str]]:
    """
    Search through the course notes.
    
    Args:
        query: The search term to look for
    
    Returns:
        Dictionary containing matching content sections
    """
    file_path = "/Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly-openai-agents-sdk/presentation/presentation_md.md"    
    
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        
    return content

In [28]:
# Create the Agent with this custom tool
from agents import Agent, Runner, trace



# Create the agent with our custom tool
agent = Agent(
    name="OpewnAI Agents SDKCourse Assistant",
    instructions="""You are a helpful assistant that searches through the OpenAI Agents SDK Course notes.
    When you find matches, summarize the relevant information found in the notes.
    Format your responses in a clear, organized way.""",
    tools=[search_course_notes]
)

# Example search
with trace("OpenAI Agents SDK Course search"):
    result = await Runner.run(
        agent, 
        "What is this presentation about?"
    )
    print(result.final_output)

The presentation is titled "Getting Started with OpenAI Agents SDK" and was conducted by Lucas Soares on 08/27/2025 during an O'Reilly Live Training session. 

### Agenda
1. **Evolution: From LLMs to Agents** - Exploring the progression from language models to agents.
2. **OpenAI Agents SDK Core Concepts** - Introducing the fundamental concepts.
3. **Building Your First Agent** - Instructions on creating a basic agent.
4. **Tools & Function Calling** - Extending agent capabilities with tools.
5. **MCP Tools (Model Context Protocol)** - Integrating agents with real-world data sources.
6. **Handoffs, Tracing & Context** - Managing complex workflows within agent interactions.
7. **Guardrails & Multi-Agent Systems** - Ensuring safe and scalable architectures.
8. **Hands-on Demo & Q&A** - Practical demonstrations and participant questions. 

The presentation emphasizes transitioning from simple text generation capabilities to advanced, stateful action and decision-making systems, using tool